In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

import numpy as np
import itertools


24/05/21 22:04:09 WARN Utils: Your hostname, yaozys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.30.192.72 instead (on interface en0)
24/05/21 22:04:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 22:04:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from sklearn.datasets import make_blobs

# Generate random dataset with 3 dimensions
n_samples = 100  # Number of samples
n_features = 3  # Number of dimensions
centers = 10  # Number of clusters
random_state = 42  # Random state for reproducibility

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)

In [3]:
X = X.tolist()

# 1. Construct Grid


In [4]:
# set parameters
min_pts = 5
eps = 2
dataset = X
partition_each_dim = [3 for i in range(n_features)]

# b_dataset = sc.broadcast(dataset)
# b_eps = sc.broadcast(eps)
# b_min_pts = sc.broadcast(min_pts)


In [5]:
# get the max/min bounds of each dimension
def get_minmax_by_data(dataset):
    # get the max/min bounds of each dimension
    min_max_bounds = np.concatenate(([np.min(dataset, axis=0)], [np.max(dataset, axis=0)]), axis=0).T   # (d, 2)
    return min_max_bounds

def get_minmax_by_bins(bin_ind, bin_bounds):
    min_max_bounds = []
    for i in range(len(bin_ind)):
        min_max_bounds.append((bin_bounds[i][bin_ind[i]], bin_bounds[i][bin_ind[i]+1]))
    return min_max_bounds

# get the bound of each bin
def get_bin_bounds(min_max_bounds, partition_each_dim):
    # get the bound of each bin
    # bin_bounds: list of np.array, each array contains the bounds of the bins in that dimension
    bin_bounds = []
    for i in range(len(partition_each_dim)):
        Lower_bound = min_max_bounds[i][0]
        Upper_bound = min_max_bounds[i][-1]
        dim_bins = np.linspace(Lower_bound, Upper_bound, partition_each_dim[i]+1, endpoint=True)
        bin_bounds.append(dim_bins)
    return bin_bounds

# locate the point in the partitioned space, return the index of space
def locate_point(x, bin_bounds, partition_each_dim):
    pos_list = []
    for i in range(n_features):
        # pos: the index of the bin in that dimension
        pos = np.digitize(x[i], bin_bounds[i]) - 1
        pos = min(partition_each_dim[i]-1, pos) # if the value is the max value, it should be in the last bin
        pos = max(0, pos) # if the value is the min value, it should be in the first bin
        pos_list.append(pos)
    return tuple(pos_list)


In [6]:
num_par = np.prod(partition_each_dim)
bin_bounds = get_bin_bounds(get_minmax_by_data(dataset), partition_each_dim)

rdd = sc.parallelize(dataset, num_par)
rdd = rdd.map(lambda x: (locate_point(x, bin_bounds, partition_each_dim), x))
rdd = rdd.sortByKey()
# rdd = rdd.groupByKey().map(lambda x: (x[0], list(x[1])))
rdd.glom().take(10)

[[((0, 0, 0), [-8.137053258537247, -4.216632385357437, -5.914030474195238]),
  ((0, 0, 0), [-8.234510428293452, -4.409675180688797, -3.9205463160910154]),
  ((0, 0, 1), [-7.178703520999728, -5.43000236549511, 0.048613680577736185]),
  ((0, 0, 1), [-5.278107750896421, -5.292824508766337, -0.4426964072703655]),
  ((0, 0, 1), [-8.729492752913176, -4.641341102161888, -1.4058519849395432]),
  ((0, 0, 1), [-6.357689452162939, -4.949627767728337, -2.787499575593065]),
  ((0, 0, 1), [-7.544624022800111, -4.632052340456593, -3.3260923666998776])],
 [((0, 1, 0), [-6.705135507978706, -3.2913518351255155, -3.8730595411819424]),
  ((0, 1, 0), [-5.444668546678066, -3.752125318334681, -3.9336470884612114])],
 [((0, 1, 1), [-6.318907911053417, -2.4616210636519287, 0.23047179940680118]),
  ((0, 1, 1), [-5.475511008607851, -3.7010613966790418, -0.7506101460672308]),
  ((0, 1, 1), [-7.917792252577944, -3.7132876011494087, -1.8981290806968292]),
  ((0, 1, 1), [-9.23526537361677, -3.970652714526209, -3.334

In [7]:
# get data with key (0, 0, 0)

test_key = (0, 0, 0)
test_value = rdd.filter(lambda x: x[0] == (0, 0, 0)).map(lambda x: x[1]).collect()
test_value

[[-8.137053258537247, -4.216632385357437, -5.914030474195238],
 [-8.234510428293452, -4.409675180688797, -3.9205463160910154]]

In [8]:
# calculate grid number for each dimension in one partition
def cal_grid_num(min_max_bounds, eps):
    global n_features
    grid_side_len = eps/np.sqrt(n_features)
    gridnum_each_dim = []
    for i in range(n_features):
        gridnum_each_dim.append(int((min_max_bounds[i][1] - min_max_bounds[i][0]) / grid_side_len))
    return gridnum_each_dim

def get_grid_each_partition(pids, bin_bounds, eps):
    grid_bin_bounds = {}
    gridnum_each_dim = {}
    for pid in pids:
        gridnum_each_dim[pid] = cal_grid_num(get_minmax_by_bins(pid, bin_bounds), eps)
        grid_bin_bounds[pid] = get_bin_bounds(get_minmax_by_bins(pid, bin_bounds), gridnum_each_dim[pid])
    return grid_bin_bounds, gridnum_each_dim

pids = rdd.keys().distinct().collect()
grid_bin_bounds, gridnum_each_dim = get_grid_each_partition(pids, bin_bounds, eps)
print(grid_bin_bounds[(0,0,0)])

[array([-10.75709897,  -9.47024932,  -8.18339967,  -6.89655002,
        -5.60970037,  -4.32285072]), array([-10.99016118,  -9.82152524,  -8.65288931,  -7.48425337,
        -6.31561744,  -5.1469815 ,  -3.97834557]), array([-10.28518036,  -9.12816924,  -7.97115811,  -6.81414699,
        -5.65713587,  -4.50012474,  -3.34311362])]


In [9]:
def find_grid(pa):
    l = []
    for pid, value in pa:
        loc = locate_point(value, grid_bin_bounds[pid], gridnum_each_dim[pid])
        l.append((pid, loc, value))
    return l

rdd_with_grid = rdd.mapPartitions(find_grid)
rdd_with_grid.collect()

[((0, 0, 0),
  (2, 5, 3),
  [-8.137053258537247, -4.216632385357437, -5.914030474195238]),
 ((0, 0, 0),
  (1, 5, 5),
  [-8.234510428293452, -4.409675180688797, -3.9205463160910154]),
 ((0, 0, 1),
  (2, 4, 2),
  [-7.178703520999728, -5.43000236549511, 0.048613680577736185]),
 ((0, 0, 1),
  (4, 4, 2),
  [-5.278107750896421, -5.292824508766337, -0.4426964072703655]),
 ((0, 0, 1),
  (1, 5, 1),
  [-8.729492752913176, -4.641341102161888, -1.4058519849395432]),
 ((0, 0, 1),
  (3, 5, 0),
  [-6.357689452162939, -4.949627767728337, -2.787499575593065]),
 ((0, 0, 1),
  (2, 5, 0),
  [-7.544624022800111, -4.632052340456593, -3.3260923666998776]),
 ((0, 1, 0),
  (3, 0, 5),
  [-6.705135507978706, -3.2913518351255155, -3.8730595411819424]),
 ((0, 1, 0),
  (4, 0, 5),
  [-5.444668546678066, -3.752125318334681, -3.9336470884612114]),
 ((0, 1, 1),
  (3, 1, 3),
  [-6.318907911053417, -2.4616210636519287, 0.23047179940680118]),
 ((0, 1, 1),
  (4, 0, 2),
  [-5.475511008607851, -3.7010613966790418, -0.7506101

# 2. Identify Core Point and Core cell

In [13]:
# find core points in each partition
def find_core_points(pa, eps, min_pts):
    for pid, gid, value in pa:
        count = 0
        for _, _, value2 in pa:
            if np.linalg.norm(np.array(value) - np.array(value2)) <= eps:
                count += 1
            if count >= min_pts:
                yield ((pid, gid), value, 1)
                break
        yield ((pid, gid), value, 0)

core_points = rdd_with_grid.mapPartitions(lambda x: find_core_points(x, eps, min_pts))
core_points.collect()

[(((0, 0, 0), (2, 5, 3)),
  [-8.137053258537247, -4.216632385357437, -5.914030474195238],
  0),
 (((0, 0, 0), (1, 5, 5)),
  [-8.234510428293452, -4.409675180688797, -3.9205463160910154],
  0),
 (((0, 0, 1), (2, 4, 2)),
  [-7.178703520999728, -5.43000236549511, 0.048613680577736185],
  0),
 (((0, 0, 1), (4, 4, 2)),
  [-5.278107750896421, -5.292824508766337, -0.4426964072703655],
  0),
 (((0, 0, 1), (1, 5, 1)),
  [-8.729492752913176, -4.641341102161888, -1.4058519849395432],
  0),
 (((0, 0, 1), (3, 5, 0)),
  [-6.357689452162939, -4.949627767728337, -2.787499575593065],
  0),
 (((0, 0, 1), (2, 5, 0)),
  [-7.544624022800111, -4.632052340456593, -3.3260923666998776],
  0),
 (((0, 1, 0), (3, 0, 5)),
  [-6.705135507978706, -3.2913518351255155, -3.8730595411819424],
  0),
 (((0, 1, 0), (4, 0, 5)),
  [-5.444668546678066, -3.752125318334681, -3.9336470884612114],
  0),
 (((0, 1, 1), (3, 1, 3)),
  [-6.318907911053417, -2.4616210636519287, 0.23047179940680118],
  1),
 (((0, 1, 1), (3, 1, 3)),
  [-

In [25]:
# get the core cells

core_cells = core_points.map(lambda x: (x[0], x[2])).reduceByKey(lambda x, y: x+y).filter(lambda x: x[1] > 0)
point_in_core_cells = core_points.join(core_cells).map(lambda x: (x[0], x[1][0]))
point_in_core_cells.collect()

[(((0, 1, 1), (4, 0, 2)),
  [-5.475511008607851, -3.7010613966790418, -0.7506101460672308]),
 (((0, 1, 1), (4, 0, 2)),
  [-5.475511008607851, -3.7010613966790418, -0.7506101460672308]),
 (((0, 2, 1), (1, 3, 3)),
  [-9.25897307940137, 6.980808398971933, 1.2200229656425567]),
 (((0, 2, 1), (1, 3, 3)),
  [-9.25897307940137, 6.980808398971933, 1.2200229656425567]),
 (((0, 2, 1), (1, 3, 3)),
  [-8.83321429999355, 7.088935782123556, 0.6069294928137614]),
 (((0, 2, 1), (1, 3, 3)),
  [-8.83321429999355, 7.088935782123556, 0.6069294928137614]),
 (((1, 2, 2), (1, 5, 0)),
  [-2.3378293418627796, 8.89863784581008, 4.338775140638813]),
 (((1, 2, 2), (1, 5, 0)),
  [-2.3378293418627796, 8.89863784581008, 4.338775140638813]),
 (((2, 0, 0), (2, 3, 3)),
  [5.759338386382913, -6.569028071399915, -6.440602365272092]),
 (((2, 0, 0), (2, 3, 3)),
  [5.759338386382913, -6.569028071399915, -6.440602365272092]),
 (((2, 0, 0), (2, 3, 3)),
  [5.411902105130354, -7.073674399518753, -5.84155909024109]),
 (((1, 2, 2

# 3. Find eps-neighbor cell pair

In [ ]:
# pass this step first

# 4. Create Graph

In [21]:
# build GraphFrame for each partition
from pyspark.sql import SparkSession
from graphframes import GraphFrame


In [27]:
spark = SparkSession.builder.appName("example").getOrCreate()


In [31]:
pid = ''.join(map(str, (0,1,1)))
pid

'011'

In [33]:
def reindex(pa):
    for (pid, gid), value in pa:
        pid = ''.join(map(str, pid))
        gid = ''.join(map(str, gid))
        yield pid+gid, value
        
point_in_core_cells = point_in_core_cells.mapPartitions(reindex)


In [34]:
schema = "ID: STRING, features: ARRAY<FLOAT>"
v = point_in_core_cells.toDF(schema)
v.show()

+------+--------------------+
|    ID|            features|
+------+--------------------+
|011402|[-5.475511, -3.70...|
|011402|[-5.475511, -3.70...|
|021133|[-9.258973, 6.980...|
|021133|[-9.258973, 6.980...|
|021133|[-8.833215, 7.088...|
|021133|[-8.833215, 7.088...|
|122150|[-2.3378294, 8.89...|
|122150|[-2.3378294, 8.89...|
|200233|[5.7593384, -6.56...|
|200233|[5.7593384, -6.56...|
|200233|[5.411902, -7.073...|
|122040|[-3.9877195, 8.29...|
|122040|[-3.9877195, 8.29...|
|011203|[-7.4028025, -3.4...|
|011203|[-7.4028025, -3.4...|
|120322|[0.28456342, 5.71...|
|120322|[0.28456342, 5.71...|
|122140|[-2.1851137, 8.62...|
|122140|[-2.1851137, 8.62...|
|202123|[4.3889117, -8.28...|
+------+--------------------+
only showing top 20 rows



# 5. Compute Connect Component

# 6. Assign the border points to clusters